In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt

In [ ]:
#Begin with SMC algorithm targeting p_theta

#Generate artificial data
A = [[np.cos(6),np.sin(6)],[-1*np.sin(6),np.cos(6)]]
C = [[0.5,.25]]
x0 = np.random.multivariate_normal([1,1], [[7/4,5/4],[5/4,7/4]], 1).T
nIters = 1000

def sampleParticle(theta):
    x0 = np.random.multivariate_normal([1,1], [[7/4,5/4],[5/4,7/4]], 1).T
    xs = []
    ys = []
    xs.append(x0)
    for i in range(5):
        eps = [[np.random.normal(0,.01,1)[0]],[np.random.normal(0,.01,1)[0]]]
        res = np.matmul(A,xs[i]) + theta[i] + eps
        xs.append(np.copy(res))
        yres = np.matmul(C,res)[0,0] + np.random.normal(0,.01,1)[0]
        ys.append(np.copy(yres))
    return xs, ys

def SMC(n,theta): #A very basic SMC algorithm
    particleTimes = [[],[],[],[],[],[]]
    for i in range(6):
        particleTimes[i] = [[0,0]]
    means = []
    covs = []
    for i in range(n):
        xs,ys = sampleParticle(theta)
        #print(xs[0][0][0])
        for j in range(6):
            particleTimes[j] = np.vstack([particleTimes[j], [xs[j][0][0],xs[j][1][0]]])
    for t in range(6):
        particleTimes[t] = np.delete(particleTimes[t], (0), axis=0)
        mean = np.mean(particleTimes[t], axis=0)
        cov = np.cov(particleTimes[t], rowvar=0)
        means.append(mean)
        covs.append(cov)
    return means,covs

In [ ]:
#PIMH
nIters = 1000
from scipy.stats import norm

#Step 1
phatMargMean = [] #These phat variables represent the MLE at all 6 time steps.
phatMargVar = []
theta = [[[7],[2]],[[5],[5]],[[-1],[2]],[[-1],[-2]],[[1],[-3]]]
means, covs = SMC(nIters,theta)
margMeans = []
margVars = []

for j in range(6):
    m = means[j][1]
    phatMargMean.append(m)
    cov = covs[j][1,1]
    phatMargVar.append(cov)

#Step 2
for i in range(nIters): #We'll do 1000 iterations of the PIMH algorithm. First 900 are burn-in
    #Part a
    meanStar, covStar = SMC(nIters,theta)
    phatMargMeanStar = [] #These phat variables represent the MLE at all 6 time steps.
    phatMargVarStar = []
    #print(phatMargMean,phatMargMeanStar)
    for j in range(6):
        m = meanStar[j][1]
        phatMargMeanStar.append(m)
        cov = covStar[j][1,1]
        phatMargVarStar.append(cov)
    #Part b
    ratio = 1
    for k in range(nIters):
        xs,ys = sampleParticle(theta)
        for n in range(6):
            ratio = ratio*norm.pdf(xs[n][1][0],loc=phatMargMeanStar[n],scale=phatMargVarStar[n])
            ratio = ratio/norm.pdf(xs[n][1][0],loc=phatMargMean[n],scale=phatMargVar[n])
    pAccept = min(1,ratio)
    accRNG = np.random.uniform(0,1,1)
    if accRNG < pAccept:
        phatMargMean = phatMargMeanStar
        phatMargVar = phatMargVarStar
        print("accepted!")
    print(i)
    if i >= 900: #Most recent 100 runs are kept
        margMeans.append(np.copy(phatMargMean))
        margVars.append(np.copy(phatMargVar))
print(phatMargMean,phatMargVar)


In [ ]:
print(margMeans)

In [ ]:
#PMMH
from scipy.stats import multivariate_normal
nIters = 5000
thetas = []
realTheta = [[[7.0],[2.0]],[[5.0],[5.0]],[[-1.0],[2.0]],[[-1.0],[-2.0]],[[1.0],[-3.0]]]

def printTheta(theta): #This function turns theta from an array of arrays to a single 10d array for easier analysis.
    allstuff = []
    for i in range(5):
        for j in range(2):
            allstuff.append(theta[i][j][0])
    return allstuff
        

#Iteration i = 0
theta = [[[2.0],[2.0]],[[2.0],[2.0]],[[2.0],[2.0]],[[2.0],[2.0]],[[2.0],[2.0]]] #This is entirely arbitrary.
means, covs = SMC(nIters,theta)

for i in range(nIters): #We'll do nIters iterations of the PMMH algorithm as an example
    #Sample theta* from q
    meanReg, covReg= SMC(50,theta)
    thetaStar = np.copy(theta)
    for j in range(len(thetaStar)):
        thetaStar[j][0][0] = thetaStar[j][0][0] + (np.random.sample()*6 - 3)
        thetaStar[j][1][0] = thetaStar[j][1][0] + (np.random.sample()*6 - 3)
    meanStar, covStar = SMC(20,thetaStar)
    ratio = 1
    for k in range(10):
        xs,ys = sampleParticle(realTheta) 
        for n in range(6): 
            #Notice that the q function is constant for our uniform dist, so the acceptance probability just gets
            #multiplied by 1.
            ratio = ratio*multivariate_normal.pdf([xs[n][0][0],xs[n][1][0]], mean=meanStar[n], cov=covStar[n])
            ratio = ratio/multivariate_normal.pdf([xs[n][0][0],xs[n][1][0]], mean=meanReg[n], cov=covReg[n])
    pAccept = min(1,ratio)
    accRNG = np.random.uniform(0,1,1)
    if accRNG < pAccept:
        theta = thetaStar
    thetas.append(printTheta(theta))
    if i%100 == 0:
        print(i)

In [ ]:
import matplotlib

rTheta = printTheta(realTheta)

#Plot output for a particular parameter in theta.
i = 0
thetaI = []
actTheta = []
for j in range(len(thetas)):
    thetaI.append(thetas[j][i])
    actTheta.append(rTheta[i])
titleStr = "Theta value of parameter " + str(i)
plt.title(titleStr)
plt.plot(thetaI,label = "Model")
plt.plot(actTheta,label = "Actual")
plt.legend()

In [ ]:
print(np.random.sample())

In [ ]:
#Plot moving average over time.
i = 3
thetaI = []
thetaTrend = []
actTheta = []
for j in range(len(thetas)):
    thetaI.append(thetas[j][i])
    actTheta.append(rTheta[i])
for j in range(len(thetaI)):
    thetaTrend.append(np.average(thetaI[0:j]))
titleStr = "Avg. Theta value of parameter " + str(i)
plt.title(titleStr)
plt.plot(thetaTrend,label = "Model")
plt.plot(actTheta,label = "Actual")
plt.legend()